In [3]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import torch.nn.functional as F
import numpy as np
learning_rate = 1e-4 # 学习率
keep_prob_rate = 0.7 #
max_epoch = 3 # 最大训练轮数
BATCH_SIZE = 50 # 每批次训练的样本数 

# 如果mnist文件夹不存在或为空，则下载数据集
DOWNLOAD_MNIST = False
if not(os.path.exists('./mnist/')) or not os.listdir('./mnist/'):
    DOWNLOAD_MNIST = True 

# 加载训练数据
train_data = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=torchvision.transforms.ToTensor(), download=DOWNLOAD_MNIST)
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
# 加载测试数据
test_data = torchvision.datasets.MNIST(root='./mnist/', train=False)
with torch.no_grad():
    test_x = Variable(torch.unsqueeze(test_data.data, dim=1)).type(torch.FloatTensor)[:500] / 255.  # 归一化处理
test_y = test_data.targets[:500].numpy()

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                # patch 7*7 ; 1 in channels ; 32 out channels ; ; stride is 1
                # padding style is same(that means the convolution opration's input and output have the same size)
                in_channels=1      ,  
                out_channels=32     ,
                kernel_size=5      ,
                stride=1           ,
                padding=2          ,
            ),
            nn.ReLU(),        # 激活函数 ReLU
            nn.MaxPool2d(2),  # 最大池化层，2x2池化
        )
        self.conv2 = nn.Sequential(
            # line 1 : convolution function, patch 5*5, 32 in channels; 64 out channels; padding style is same; stride is 1
            # line 2 : choosing your activation function
            # line 3 : pooling operation function.
            nn.Conv2d(  # 32个输入通道，64个输出通道
                in_channels=32,
                out_channels=64,
                kernel_size=5,
                stride=1,
                padding=2,
            ),
            nn.ReLU(),        # 激活函数 ReLU
            nn.MaxPool2d(2),  # 最大池化层，2x2池化
        )
        self.out1 = nn.Linear(7*7*64, 1024, bias=True)  # 全连接层1
        self.dropout = nn.Dropout(keep_prob_rate)       # Dropout层
        self.out2 = nn.Linear(1024, 10, bias=True)      # 输出层，10个类别对应MNIST数字

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(-1, 7 * 7 * 64)  # 将卷积层的输出展平为1D向量(batch_size, 7*7*64)
        out1 = self.out1(x)
        out1 = F.relu(out1)
        out1 = self.dropout(out1)
        out2 = self.out2(out1)
        output = F.softmax(out2, dim=1)
        return output


def test(cnn):
    global prediction
    y_pre = cnn(test_x)
    _,pre_index= torch.max(y_pre,1)
    pre_index= pre_index.view(-1)
    prediction = pre_index.data.numpy()
    correct  = np.sum(prediction == test_y)
    return correct / 500.0


def train(cnn):
    optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate )
    loss_func = nn.CrossEntropyLoss()
    for epoch in range(max_epoch):
        for step, (x_, y_) in enumerate(train_loader):
            x ,y= Variable(x_),Variable(y_)
            output = cnn(x)  
            loss = loss_func(output,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if step != 0 and step % 20 ==0:
                print("="*10, step, "="*5, "="*5, "test accuracy is ", test(cnn), "="*10)

if __name__ == '__main__':
    cnn = CNN()
    train(cnn)




========== 20 ===== ===== test accuracy is  0.29 ==========
========== 40 ===== ===== test accuracy is  0.276 ==========
========== 60 ===== ===== test accuracy is  0.46 ==========
========== 80 ===== ===== test accuracy is  0.562 ==========
========== 100 ===== ===== test accuracy is  0.642 ==========
========== 120 ===== ===== test accuracy is  0.632 ==========
========== 140 ===== ===== test accuracy is  0.704 ==========
========== 160 ===== ===== test accuracy is  0.728 ==========
========== 180 ===== ===== test accuracy is  0.716 ==========
========== 200 ===== ===== test accuracy is  0.792 ==========
========== 220 ===== ===== test accuracy is  0.802 ==========
========== 240 ===== ===== test accuracy is  0.824 ==========
========== 260 ===== ===== test accuracy is  0.848 ==========
========== 280 ===== ===== test accuracy is  0.87 ==========
========== 300 ===== ===== test accuracy is  0.868 ==========
========== 320 ===== ===== test accuracy is  0.858 ==========
========== 340 